In [ ]:
import pandas as pd
import numpy as np
import joblib as jb

import tensorflow
import keras
import matplotlib.pyplot as plt
from keras import callbacks
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, MaxPool2D, UpSampling2D

from keras.engine.topology import Layer, InputSpec
from keras import backend as k
from keras import regularizers

In [ ]:
# load preprocessed dragonfly images
trainpath = r'D:\Linnaeus_models\dragon\train\dragon_train.npy'
testpath = r'D:\Linnaeus_models\dragon\test\dragon_test.npy'
dragons = np.concatenate((np.load(trainpath), np.load(testpath)), axis=0)
print(f'Number of images: {len(dragons)}')

##  Auto Encoding Using CNN

In [ ]:
# separate into training set and validation set and reshape to fit the NN
x_train = dragons[0:16000] 
x_val = dragons[16000:]

In [ ]:
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)

In [ ]:
# this is our input placeholder
input_img = Input(shape=(256, 256, 1))

# "encoded" is the encoded representation of the input
encoded = Conv2D(48, (3, 3), activation='relu', padding='same')(input_img)
encoded = MaxPool2D((2, 2), padding='same')(encoded)
encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPool2D((2, 2), padding='same')(encoded)
encoded = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPool2D((2, 2), padding='same')(encoded)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPool2D((2, 2), padding='same')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(16, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(32, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(48, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(1, (3, 3), padding='same')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [ ]:
autoencoder.summary()

In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='Adam', loss='mse')

train_history = autoencoder.fit(x_train, x_train, epochs=11, batch_size=100, 
                                validation_data=(x_val, x_val))

In [ ]:
preds = autoencoder.predict(x_val)

In [ ]:
reverted_x = np.squeeze(x_val, axis=-1)

In [ ]:
plt.imshow(reverted_x[2].reshape(256, 256), cmap='gray')

In [ ]:
plt.imshow(x_val[2].reshape(256, 256), cmap='gray')

In [ ]:
plt.imshow(preds[2].reshape(256, 256), cmap='gray')

In [ ]:
# save model
model_name =  r'D:\Linnaeus_models\dragon_reconstruction_v2.pkl'
jb.dump(autoencoder, model_name)

In [ ]:
# past models